In [2]:
# importar bibliotecas de software

import pandas as pd
import numpy as np
import musicbrainzngs
from tqdm.notebook import tqdm

## Exploração de dados

In [42]:
charts = pd.read_csv("charts.csv")
charts.dtypes

Unnamed: 0         int64
Unnamed: 0.1       int64
rank               int64
previous_week     object
artist            object
album             object
label             object
weeks_on_chart     int64
calendar_week      int64
year               int64
dtype: object

In [43]:
charts = charts.drop(['Unnamed: 0', "Unnamed: 0.1"], axis=1)
charts

,rank,previous_week,artist,album,label,weeks_on_chart,calendar_week,year
0,1,1,Rui Veloso,O concerto acústico,Virgin,8,1,2004
1,2,3,Robbie Williams,Live Summer 2003,Chrysalis,14,1,2004
2,3,2,Tribalistas,Tribalistas,Emi,42,1,2004
3,4,4,Linkin Park,Live In Texas,Wb,7,1,2004
4,5,6,Tony Carreira,Ao vivo no pavilhao atlantico,Espacial,22,1,2004
...,...,...,...,...,...,...,...,...
24805,26,35,Pink Floyd,The Wall,Warner,64,52,2019
24806,27,32,António Variações,O melhor de António Variações,Warner,23,52,2019
24807,28,22,João Pedro Pais,Confidências (de um homem vulgar),Sony Music,10,52,2019
24808,29,23,Camila Cabello,Romance,Sony Music,3,52,2019


In [44]:
charts.isna().sum()

rank                 0
previous_week     2475
artist              55
album                0
label               57
weeks_on_chart       0
calendar_week        0
year                 0
dtype: int64

In [131]:
charts["artist"].value_counts()

Mariza                     968
Tony Carreira              641
Ana Moura                  616
António Zambujo            412
Soundtrack                 336
                          ... 
Pentatonix                   1
Warpaint                     1
Panic! At The Disco          1
Hermínia Silva               1
Pat Metheny Unity Group      1
Name: artist, Length: 1114, dtype: int64

In [3]:
# lista de artistas
unique_artists = list(charts["artist"].unique())

NameError: name 'charts' is not defined

## Ir buscar nacionalidade dos artistas via MusicBrainz

In [32]:
musicbrainzngs.auth()
musicbrainzngs.set_useragent("chart", 0.1, "https://rute.radio")

In [82]:
new_df = pd.DataFrame()

for artist in tqdm(unique_artists):
    artist_list = musicbrainzngs.search_artists(artist=artist)
    artist_df = pd.DataFrame.from_records(artist_list)
    new_df = pd.concat([new_df, artist_df])
    
print("Já está")


Done!!!


In [83]:
#retirar informação dos dicionários extraídos do musicbrainz


def get_name(x):
    if type(x) == dict:
        return x.get("name")
    
    return "---"

def get_country(x):
    if type(x) == dict:
        return x.get("country")
    
    return "---"
            
new_df["artist"] = new_df["artist-list"].apply(get_name)  
new_df["country"] = new_df["artist-list"].apply(get_country)
new_df

,artist-count,artist-list,artist,country
0,100,"{'id': '758416ad-4871-472a-ab82-fd6e1b7b6c67',...",Rui Veloso,PT
1,100,"{'id': 'f07dbc2f-317b-470f-bad4-5f1b0eb6faf1',...",Caetano Veloso,None
2,100,"{'id': 'b4082b3e-1e45-4544-8d5b-a5fc367648fa',...",Moreno Veloso,BR
3,100,"{'id': '55536771-cd1b-455a-afb4-ed3a4b197c8b',...",RUI,None
4,100,"{'id': 'f1ad854a-3b6f-4f57-b902-a29baa2c9efa',...",Rui,None
...,...,...,...,...
21,91647,"{'id': 'e6181aae-1216-4f8f-9fe3-79222759b42f',...",The Reggae Philharmonic Orchestra,None
22,91647,"{'id': '70a803e5-15db-41bb-861d-ec9af670f323',...",Buffalo Philharmonic Orchestra,US
23,91647,"{'id': '92fc1517-7e49-44a0-9ca2-644e35b424c5',...",BBC Philharmonic,None
24,91647,"{'id': '8bb66e3d-cf3a-4186-a844-fb1bc7367fcb',...",The Royal Philharmonic Chamber Ensemble,None


In [84]:
new_df = new_df.drop(['artist-list', "artist-count"], axis=1)

In [91]:
new_df.reset_index(inplace=True)
new_df.drop("index", axis=1, inplace = True)

In [162]:
new_df.drop_duplicates(subset="artist", keep = "first", inplace=True)
new_df

,artist,country
0,Rui Veloso,PT
1,Caetano Veloso,None
2,Moreno Veloso,BR
3,RUI,None
4,Rui,None
...,...,...
23119,Muri & Mario,DK
23138,Sara Hickman,US
23142,Sara Gazarek,US
23169,The Royal Philharmonic Chamber Ensemble,None


In [136]:
clean_artists  = new_df["artist"].isin(unique_artists)
clean_artists

0         True
1         True
2        False
3        False
4        False
         ...  
23166    False
23167    False
23168    False
23169    False
23170     True
Name: artist, Length: 23171, dtype: bool

In [168]:
newish = charts.merge(new_df, how="left", on='artist')

#newish.drop_duplicates(inplace= True)
#newish.reset_index(inplace = True)

newish

,rank,previous_week,artist,album,label,weeks_on_chart,calendar_week,year,country
0,1,1,Rui Veloso,O concerto acústico,Virgin,8,1,2004,PT
1,2,3,Robbie Williams,Live Summer 2003,Chrysalis,14,1,2004,GB
2,3,2,Tribalistas,Tribalistas,Emi,42,1,2004,BR
3,4,4,Linkin Park,Live In Texas,Wb,7,1,2004,US
4,5,6,Tony Carreira,Ao vivo no pavilhao atlantico,Espacial,22,1,2004,PT
...,...,...,...,...,...,...,...,...,...
24805,26,35,Pink Floyd,The Wall,Warner,64,52,2019,GB
24806,27,32,António Variações,O melhor de António Variações,Warner,23,52,2019,PT
24807,28,22,João Pedro Pais,Confidências (de um homem vulgar),Sony Music,10,52,2019,PT
24808,29,23,Camila Cabello,Romance,Sony Music,3,52,2019,US


In [169]:
newish.sort_values(by=['calendar_week', 'year', "rank"], inplace=True)
#newish.drop_duplicates(subset="country")
newish

,rank,previous_week,artist,album,label,weeks_on_chart,calendar_week,year,country
0,1,1,Rui Veloso,O concerto acústico,Virgin,8,1,2004,PT
1,2,3,Robbie Williams,Live Summer 2003,Chrysalis,14,1,2004,GB
2,3,2,Tribalistas,Tribalistas,Emi,42,1,2004,BR
3,4,4,Linkin Park,Live In Texas,Wb,7,1,2004,US
4,5,6,Tony Carreira,Ao vivo no pavilhao atlantico,Espacial,22,1,2004,PT
...,...,...,...,...,...,...,...,...,...
24805,26,35,Pink Floyd,The Wall,Warner,64,52,2019,GB
24806,27,32,António Variações,O melhor de António Variações,Warner,23,52,2019,PT
24807,28,22,João Pedro Pais,Confidências (de um homem vulgar),Sony Music,10,52,2019,PT
24808,29,23,Camila Cabello,Romance,Sony Music,3,52,2019,US


In [170]:
newish.reset_index(inplace = True)
newish.drop("index", axis=1, inplace = True)


In [171]:
newish.head(55)

,rank,previous_week,artist,album,label,weeks_on_chart,calendar_week,year,country
0,1,1,Rui Veloso,O concerto acústico,Virgin,8,1,2004,PT
1,2,3,Robbie Williams,Live Summer 2003,Chrysalis,14,1,2004,GB
2,3,2,Tribalistas,Tribalistas,Emi,42,1,2004,BR
3,4,4,Linkin Park,Live In Texas,Wb,7,1,2004,US
4,5,6,Tony Carreira,Ao vivo no pavilhao atlantico,Espacial,22,1,2004,PT
5,6,11,Red Hot Chili Peppers,Greatest Hits,Wb,7,1,2004,US
6,7,5,Ennio Morricone & Dulce Pontes,Focus,Polydor,11,1,2004,NaN
7,8,7,Maria Rita,Maria Rita,Wmb,9,1,2004,BR
8,9,9,Luís Represas,Fora de mão,Mercury,10,1,2004,PT
9,10,10,Julio Iglesias,Divorcio,Columbia,10,1,2004,ES


In [172]:
newish["artist"].value_counts()

Mariza                                                 968
Tony Carreira                                          641
Ana Moura                                              616
António Zambujo                                        412
Soundtrack                                             336
                                                      ... 
Panda e os caricas                                       1
Mac DeMarco                                              1
Red Hot Chili Peppers                                    1
Coro Gulkbenkian, Orquestra Metropolitana De Lisboa      1
Pat Metheny Unity Group                                  1
Name: artist, Length: 1114, dtype: int64

In [191]:
newish.head()

,rank,previous_week,artist,album,label,weeks_on_chart,calendar_week,year,country
0,1,1,Rui Veloso,O concerto acústico,Virgin,8,1,2004,PT
1,2,3,Robbie Williams,Live Summer 2003,Chrysalis,14,1,2004,GB
2,3,2,Tribalistas,Tribalistas,Emi,42,1,2004,BR
3,4,4,Linkin Park,Live In Texas,Wb,7,1,2004,US
4,5,6,Tony Carreira,Ao vivo no pavilhao atlantico,Espacial,22,1,2004,PT


In [196]:
fix_nation = newish.loc[newish['country'].isnull()]
fix_nation

,rank,previous_week,artist,album,label,weeks_on_chart,calendar_week,year,country
6,7,5,Ennio Morricone & Dulce Pontes,Focus,Polydor,11,1,2004,NaN
18,19,20,Frei Hermano Da Camara,"Vivo d'arte, vivo d'amor",Farol Musica,3,1,2004,NaN
31,2,1,Seal,Best 1991-2004,Wb,7,1,2005,None
38,9,14,Jay-Z / Linkin Park,Collision Course,Wbr,3,1,2005,NaN
54,25,23,Eminem,Encore,Interscope,8,1,2005,None
...,...,...,...,...,...,...,...,...,...
24787,8,7,Carlos Do Carmo,Oitenta,Universal,18,52,2019,NaN
24794,15,11,André Rieu And His Johann Strauss Orchestra,Happy Days,Universal,5,52,2019,NaN
24801,22,25,Panda e Os Caricas,Era uma vez... Panda e Os Caricas,Universal,23,52,2019,None
24802,23,34,Panda e Os Caricas,Brinca e aprende com o Panda e Os Caricas,Universal,2,52,2019,None


In [199]:
#adicionar nacionalidades manualmente

newish.loc[newish['artist'] == 'Caetano Veloso', 'country'] = 'BR'
newish.loc[newish['artist'] == 'Martinho Da Vila', 'country'] = 'BR'
newish.loc[newish['artist'] == 'Marco Paulo', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Delfins', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Os Patinhos', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Frei Hermano Da Camara', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Zeze Di Camargo & Luciano', 'country'] = 'BR'
newish.loc[newish['artist'] == 'Jose Mário Branco', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Panda e Os Caricas', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Carlos Do Carmo', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Eminem', 'country'] = 'US'

In [200]:
fix_nation = newish.loc[newish['country'].isnull()]
fix_nation

,rank,previous_week,artist,album,label,weeks_on_chart,calendar_week,year,country
6,7,5,Ennio Morricone & Dulce Pontes,Focus,Polydor,11,1,2004,NaN
31,2,1,Seal,Best 1991-2004,Wb,7,1,2005,None
38,9,14,Jay-Z / Linkin Park,Collision Course,Wbr,3,1,2005,NaN
60,1,1,D'ZRT,Ao vivo no Coliseus,Farol Musica,4,1,2006,NaN
72,13,17,Dire Straits & Mark Knopfler,Private Investigations - The Best Of,Mercury,8,1,2006,NaN
...,...,...,...,...,...,...,...,...,...
24777,28,NaN,Zayn,Icarus Falls,Sony Music,1,52,2018,NaN
24782,3,5,Camané & Mário Laginha,Aqui está-se sossegado,Warner,6,52,2019,NaN
24783,4,4,Elas,9,Sony Music,4,52,2019,None
24786,7,9,Nick Cave & The Bad Seeds,Ghosteen,Popstock,8,52,2019,NaN


In [201]:
newish.loc[newish['artist'] == 'Seal', 'country'] = 'GB'
newish.loc[newish['artist'] == 'Nick Cave & The Bad Seeds', 'country'] = 'AU'
newish.loc[newish['artist'] == 'Elas', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Camané & Mário Laginha', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Zayn', 'country'] = 'GB'
newish.loc[newish['artist'] == 'André Rieu And His Johann Strauss Orchestra', 'country'] = 'NL'
newish.loc[newish['artist'] == 'Dire Straits & Mark Knopfler', 'country'] = 'GB'
newish.loc[newish['artist'] == 'D\'ZRT', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Jay-Z / Linkin Park', 'country'] = 'US'
newish.loc[newish['artist'] == 'Ennio Morricone & Dulce Pontes', 'country'] = 'IT/PT'


In [202]:
fix_nation = newish.loc[newish['country'].isnull()]
fix_nation

,rank,previous_week,artist,album,label,weeks_on_chart,calendar_week,year,country
80,21,6,Diana Krall feat. The Clayton/Hamilton Jazz Or...,Christmas Songs,Verve,9,1,2006,NaN
90,1,1,4 Taste,4 Taste,Farol Musica,0,1,2007,NaN
91,2,2,André Sardet,Acústico,Farol Musica,0,1,2007,None
93,4,5,Floribella,O melhor natal,Som Livre,0,1,2007,None
102,13,15,Docemania,Docemania,Universal,0,1,2007,NaN
...,...,...,...,...,...,...,...,...,...
24754,5,4,Soundtrack / Lady Gaga / Bradley Cooper,A Star Is Born,Universal,12,52,2018,NaN
24763,14,19,António Zambujo & Miguel Araújo,28 noites ao vivo nos coliseus,Universal,29,52,2018,NaN
24764,15,16,Rosebonbon,Rosebonbon,Sony Music,3,52,2018,NaN
24770,21,21,Raquel Tavares,Roberto Carlos por Raquel Tavares,Sony Music,57,52,2018,None


In [203]:
newish.loc[newish['artist'] == 'Tony Bennett & Diana Krall', 'country'] = 'US'
newish.loc[newish['artist'] == '4 Taste', 'country'] = 'PT'
newish.loc[newish['artist'] == 'André Sardet', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Floribella', 'country'] = 'PT'
newish.loc[newish['artist'] == 'António Zambujo & Miguel Araújo', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Raquel Tavares', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Rosebonbon', 'country'] = 'PT'
newish.loc[newish['artist'] == 'D\'ZRT', 'country'] = 'PT'

In [205]:
fix_nation = newish.loc[newish['country'].isnull()]
fix_nation.head(30)

,rank,previous_week,artist,album,label,weeks_on_chart,calendar_week,year,country
80,21,6,Diana Krall feat. The Clayton/Hamilton Jazz Or...,Christmas Songs,Verve,9,1,2006,NaN
102,13,15,Docemania,Docemania,Universal,0,1,2007,NaN
105,16,NaN,Beto,Porto de abrigo,Zona Musica,0,1,2007,None
110,21,30,Bebé Lilly,Este e o meu mundo,Sony Bmg,0,1,2007,NaN
114,25,29,Rouxinol Faduncho,Best Of,Som Livre,0,1,2007,None
116,27,18,Sam The Kid,Pratica(mente),Edel,0,1,2007,NaN
119,30,20,Anabela & Carlos Guilherme,Encontro,Som Livre / Zona,0,1,2007,NaN
120,1,1,Just Girls,Just Girls,Farol Musica,5,1,2008,None
122,3,5,Vanessa Da Mata,SIM,Sony Bmg,22,1,2008,NaN
127,8,6,Mafalda Veiga & João Pedro Pais,Lado a lado,Som Livre / Vc,34,1,2008,NaN


In [206]:
newish.loc[newish['artist'] == 'Docemania', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Beto', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Bebé Lilly', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Rouxinol Faduncho', 'country'] = 'PT'

newish.loc[newish['artist'] == 'Sam The Kid', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Anabela & Carlos Guilherme', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Just Girls', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Mafalda Veiga & João Pedro Pais', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Caixinha de Sonhos', 'country'] = 'PT'

newish.loc[newish['artist'] == 'Panda vai à escola', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Rua da Saudade', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Leandro [PT]', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Rodrigo Leão & Cinema Ensemble', 'country'] = 'PT'


In [207]:
fix_nation = newish.loc[newish['country'].isnull()]
fix_nation.head(30)

,rank,previous_week,artist,album,label,weeks_on_chart,calendar_week,year,country
80,21,6,Diana Krall feat. The Clayton/Hamilton Jazz Or...,Christmas Songs,Verve,9,1,2006,NaN
122,3,5,Vanessa Da Mata,SIM,Sony Bmg,22,1,2008,NaN
134,15,12,Avô Cantigas,Fantasminha brincalhão,Sony Bmg,33,1,2008,None
181,2,1,Mário Branco - Sérgio Godinho - Fausto,Três cantos: Ao vivo,Emi,4,1,2010,NaN
201,22,27,Soundtrack / Michael Jackson,This Is It,Sony Music,9,1,2010,NaN
208,29,25,Soundtrack / Miley Cyrus,Hannah Montana - The Movie,Emi,32,1,2010,NaN
232,23,12,Thirty Seconds To Mars,This Is War,Emi,9,1,2011,NaN
237,28,10,Sting feat. The Royal Philharmonic Concert Orc...,Live In Berlin,Universal,5,1,2011,NaN
239,30,++,Kings Of Leon,Come Around Sundown,Sony Music,6,1,2011,NaN
276,7,12,Maria de Vasconcelos,As cançoes da Maria,Arthouse,44,1,2013,NaN


In [209]:
newish.loc[newish['artist'] == 'Panda E Os Amigos', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Paulo De Carvalho', 'country'] = 'PT'
newish.loc[newish['artist'] == 'D.A.M.A.', 'country'] = 'PT'
newish.loc[newish['artist'] == 'C4 Pedro', 'country'] = 'AN'

newish.loc[newish['artist'] == 'Calema', 'country'] = 'STP'
newish.loc[newish['artist'] == 'Jorge Palma & Sérgio Godinho', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Maria de Vasconcelos', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Pedro Abrunhosa & Comité Caviar', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Xana Toc Toc', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Virgul', 'country'] = 'PT'

newish.loc[newish['artist'] == 'Rancho De Cantadores De Aldeia Nova De São Bento', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Mário Branco - Sérgio Godinho - Fausto', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Vanessa Da Mata', 'country'] = 'BR'

newish.loc[newish['artist'] == 'Thirty Seconds To Mars', 'country'] = 'US'
newish.loc[newish['artist'] == 'Kings Of Leon', 'country'] = 'US'

In [210]:
fix_nation = newish.loc[newish['country'].isnull()]
fix_nation.head(30)

,rank,previous_week,artist,album,label,weeks_on_chart,calendar_week,year,country
80,21,6,Diana Krall feat. The Clayton/Hamilton Jazz Or...,Christmas Songs,Verve,9,1,2006,NaN
134,15,12,Avô Cantigas,Fantasminha brincalhão,Sony Bmg,33,1,2008,None
201,22,27,Soundtrack / Michael Jackson,This Is It,Sony Music,9,1,2010,NaN
208,29,25,Soundtrack / Miley Cyrus,Hannah Montana - The Movie,Emi,32,1,2010,NaN
237,28,10,Sting feat. The Royal Philharmonic Concert Orc...,Live In Berlin,Universal,5,1,2011,NaN
312,13,14,NaN,Unorthodox Jukebox,Warner,8,1,2014,NaN
389,30,37,Jardim De Infância,Jardim De Infância,Espacial,0,1,2016,NaN
423,4,6,Maria Vasconcelos,As canções da Maria - Especial história de Por...,Sony Music,9,1,2018,NaN
452,3,5,Soundtrack / Lady Gaga / Bradley Cooper,A Star Is Born,Universal,13,1,2019,NaN
462,13,4,Soundtrack / Queen,Bohemian Rhapsody,Universal,11,1,2019,NaN


In [211]:
newish.loc[newish['artist'] == 'Maria Vasconcelos', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Vasco [PT]', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Pedro Osório', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Lucy', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Avô Cantigas', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Maria de Vasconcelos', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Elis Regina & Antonio Carlos Jobim', 'country'] = 'BR'
newish.loc[newish['artist'] == 'Herois do Mar', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Vinicius De Moraes', 'country'] = 'BR'

newish.loc[newish['artist'] == 'Pigloo', 'country'] = 'FR'
newish.loc[newish['artist'] == 'Carla Bruni', 'country'] = 'FR'
newish.loc[newish['artist'] == 'Vanessa Da Mata', 'country'] = 'BR'

newish.loc[newish['artist'] == 'Guns N\' Roses', 'country'] = 'US'
newish.loc[newish['artist'] == 'Dino D\'Santiago', 'country'] = 'PT'

In [212]:
fix_nation = newish.loc[newish['country'].isnull()]
fix_nation.head(30)

,rank,previous_week,artist,album,label,weeks_on_chart,calendar_week,year,country
80,21,6,Diana Krall feat. The Clayton/Hamilton Jazz Or...,Christmas Songs,Verve,9,1,2006,NaN
201,22,27,Soundtrack / Michael Jackson,This Is It,Sony Music,9,1,2010,NaN
208,29,25,Soundtrack / Miley Cyrus,Hannah Montana - The Movie,Emi,32,1,2010,NaN
237,28,10,Sting feat. The Royal Philharmonic Concert Orc...,Live In Berlin,Universal,5,1,2011,NaN
312,13,14,NaN,Unorthodox Jukebox,Warner,8,1,2014,NaN
389,30,37,Jardim De Infância,Jardim De Infância,Espacial,0,1,2016,NaN
452,3,5,Soundtrack / Lady Gaga / Bradley Cooper,A Star Is Born,Universal,13,1,2019,NaN
462,13,4,Soundtrack / Queen,Bohemian Rhapsody,Universal,11,1,2019,NaN
475,26,42,Twenty One Pilots,Trench,Warner,12,1,2019,NaN
577,8,NaN,Os Pintainhos,Os Pintainhos,Farol Musica,0,2,2007,NaN


In [213]:
newish.loc[newish['artist'] == 'Os Pintainhos', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Márcia', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Beto & Ménito', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Soundtrack / Miley Cyrus', 'country'] = 'US'
newish.loc[newish['artist'] == 'Viky', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Tom Jobim', 'country'] = 'BR'


newish.loc[newish['artist'] == 'Soundtrack / Michael Jackson', 'country'] = 'US'
newish.loc[newish['artist'] == 'Soundtrack / Eddie Vedder', 'country'] = 'US'
newish.loc[newish['artist'] == 'Soundtrack / Pearl Jam', 'country'] = 'US'
newish.loc[newish['artist'] == 'Soundtrack / Lady Gaga / Bradley Cooper', 'country'] = 'US'
newish.loc[newish['artist'] == 'Twenty One Pilots', 'country'] = 'US'

In [214]:
fix_nation = newish.loc[newish['country'].isnull()]
fix_nation.head(30)

,rank,previous_week,artist,album,label,weeks_on_chart,calendar_week,year,country
80,21,6,Diana Krall feat. The Clayton/Hamilton Jazz Or...,Christmas Songs,Verve,9,1,2006,NaN
237,28,10,Sting feat. The Royal Philharmonic Concert Orc...,Live In Berlin,Universal,5,1,2011,NaN
312,13,14,NaN,Unorthodox Jukebox,Warner,8,1,2014,NaN
389,30,37,Jardim De Infância,Jardim De Infância,Espacial,0,1,2016,NaN
462,13,4,Soundtrack / Queen,Bohemian Rhapsody,Universal,11,1,2019,NaN
791,12,13,NaN,Unorthodox Jukebox,Warner,9,2,2014,NaN
937,8,13,Soundtrack / Queen,Bohemian Rhapsody,Universal,12,2,2019,NaN
1247,18,++,NaN,----------,NaN,0,3,2013,NaN
1248,19,++,NaN,----------,NaN,0,3,2013,NaN
1251,22,++,NaN,----------,NaN,0,3,2013,NaN


In [215]:
newish.loc[newish['artist'] == 'Ana Bacalhau', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Luís Represas e João Gil', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Escolinha de música', 'country'] = 'PT'
newish.loc[newish['artist'] == 'JP Simões', 'country'] = 'PT'
newish.loc[newish['artist'] == 'O Mundo Da Sara', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Jardim De Infância', 'country'] = 'PT'
newish.loc[newish['artist'] == 'You Can\'t Win, Charlie Brown', 'country'] = 'PT'


newish.loc[newish['artist'] == 'D\'Angelo and The Vanguard 	', 'country'] = 'US'
newish.loc[newish['artist'] == 'The XX', 'country'] = 'GB'
newish.loc[newish['artist'] == 'Soundtrack / Queen', 'country'] = 'GB'
newish.loc[newish['artist'] == 'Soundtrack / Andrew Lloyd Webber', 'country'] = 'GB'
newish.loc[newish['artist'] == 'Yo La Tengo', 'country'] = 'US'

In [216]:
fix_nation = newish.loc[newish['country'].isnull()]
fix_nation.head(30)

,rank,previous_week,artist,album,label,weeks_on_chart,calendar_week,year,country
80,21,6,Diana Krall feat. The Clayton/Hamilton Jazz Or...,Christmas Songs,Verve,9,1,2006,NaN
237,28,10,Sting feat. The Royal Philharmonic Concert Orc...,Live In Berlin,Universal,5,1,2011,NaN
312,13,14,NaN,Unorthodox Jukebox,Warner,8,1,2014,NaN
791,12,13,NaN,Unorthodox Jukebox,Warner,9,2,2014,NaN
1247,18,++,NaN,----------,NaN,0,3,2013,NaN
1248,19,++,NaN,----------,NaN,0,3,2013,NaN
1251,22,++,NaN,----------,NaN,0,3,2013,NaN
1253,24,++,NaN,----------,NaN,0,3,2013,NaN
1254,25,++,NaN,----------,NaN,0,3,2013,NaN
1258,29,++,NaN,----------,NaN,0,3,2013,NaN


In [217]:
newish.loc[newish['artist'] == 'Ana Free', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Helder Moutinho', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Manuel Fúria E Os Náufragos', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Filipe Pinto', 'country'] = 'PT'
newish.loc[newish['artist'] == 'The Legendary Tigerman', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Joka Fantasminha', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Kalu', 'country'] = 'PT'


newish.loc[newish['artist'] == 'D\'Angelo and The Vanguard', 'country'] = 'US'
newish.loc[newish['artist'] == 'The Lumineers', 'country'] = 'US'
newish.loc[newish['artist'] == 'Florence + The Machine', 'country'] = 'GB'
newish.loc[newish['artist'] == 'Joss Stone', 'country'] = 'GB'
newish.loc[newish['artist'] == 'Belle And Sebastian', 'country'] = 'GB'
newish.loc[newish['artist'] == 'Caetano Veloso / Gilberto Gil', 'country'] = 'BR'
newish.loc[newish['artist'] == 'Mallu', 'country'] = 'BR'

In [221]:
fix_nation = newish.loc[newish['country'].isnull()]
fix_nation.tail(30)

,rank,previous_week,artist,album,label,weeks_on_chart,calendar_week,year,country
23659,20,20,NaN,Unorthodox Jukebox,Warner,5,50,2013,NaN
23729,30,22,Frei Hermano Da Câmara,O nazareno,Sony Music,2,50,2015,NaN
23877,28,NaN,Soundtrack / Vangelis,Alexander,Sony Classical,1,51,2004,NaN
23885,6,5,Diana Krall feat. The Clayton/Hamilton Jazz Or...,Christmas Songs,Verve,7,51,2005,NaN
23953,14,11,Teresa Salgueiro & Lusitânia,La serena,Farol Music,8,51,2007,NaN
23966,27,25,Pedro Khima,Pedro Khima,Farol Musica,15,51,2007,None
24040,11,18,Soundtrack / Hannah Montana,Hannah Montana Forever,Emi,8,51,2010,NaN
24046,17,22,Diana Krall feat. The Clayton/Hamilton Jazz Or...,Christmas Songs,Universal,12,51,2010,NaN
24057,28,21,Sting feat. The Royal Philharmonic Concert Orc...,Live In Berlin,Universal,2,51,2010,NaN
24106,17,14,Maria João Pires / Carlos do Carmo,Maria João Pires / Carlos do Carmo,Universal,3,51,2012,NaN


In [224]:
newish.loc[newish['artist'] == 'Teresa Salgueiro & Lusitânia', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Pedro Khima', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Dead Combo E As Cordas Da Má Fama', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Frei Hermano Da Câmara', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Maria João Pires / Carlos do Carmo', 'country'] = 'PT'

newish.loc[newish['artist'] == 'Sting feat. The Royal Philharmonic Concert Orchestra', 'country'] = 'GB'
newish.loc[newish['artist'] == 'Soundtrack / Hannah Montana', 'country'] = 'US'
newish.loc[newish['artist'] == '5 Seconds Of Summer', 'country'] = 'GB'
newish.loc[newish['artist'] == 'Diana Krall feat. The Clayton/Hamilton Jazz Orchestra', 'country'] = 'US'


In [225]:
fix_nation = newish.loc[newish['country'].isnull()]
fix_nation.tail(30)

,rank,previous_week,artist,album,label,weeks_on_chart,calendar_week,year,country
22763,24,14,Soundtrack / Roger Waters,Roger Waters The Wall,Sony Music,2,48,2015,NaN
22853,24,19,Diabo Na Cruz,Lebre,Sony Music,7,48,2018,NaN
22888,29,NaN,Rod Stewart with The Royal Philharmonic Orchestra,You're In My Heart,Warner,1,48,2019,NaN
22939,20,15,System Of A Down,Hypnotize,Columbia,2,49,2005,NaN
22946,27,21,"Nat ""King"" Cole",Nat King Cole en español,Capitol,11,49,2005,NaN
23055,16,20,LazyTown,LazyTown,Iplay,2,49,2009,None
23063,24,NaN,Bossa Nossa,Mais Bossa Nossa,Sony Music,1,49,2009,NaN
23086,17,17,Rodrigo Leão & Vox Ensemble,Ave mundi luminar,Sony Music,5,49,2010,NaN
23099,30,NaN,Coro Infantil Santo Amaro de Oeiras,Já é Natal,Vidisco,1,49,2010,NaN
23129,30,17,Caetano e Maria Gadú,Multishow ao vivo,Universal,10,49,2011,NaN


In [226]:
newish.loc[newish['artist'] == 'Caetano e Maria Gadú', 'country'] = 'BR'
newish.loc[newish['artist'] == 'Diabo Na Cruz', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Ricardo Reis Pinto', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Rodrigo Leão & Vox Ensemble', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Coro Infantil Santo Amaro de Oeiras', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Tim [PT]', 'country'] = 'PT'

newish.loc[newish['artist'] == 'System Of A Down', 'country'] = 'US'
newish.loc[newish['artist'] == 'Miley Cyrus', 'country'] = 'US'
newish.loc[newish['artist'] == 'Soundtrack / Roger Waters', 'country'] = 'GB'
newish.loc[newish['artist'] == 'Nat "King" Cole', 'country'] = 'US'
newish.loc[newish['artist'] == 'Rod Stewart with The Royal Philharmonic Orchestra', 'country'] = 'GB'

In [227]:
fix_nation = newish.loc[newish['country'].isnull()]
fix_nation.tail(30)

,rank,previous_week,artist,album,label,weeks_on_chart,calendar_week,year,country
22070,21,14,Maria João & Mário Laginha,Chocolate,Universal,2,47,2008,NaN
22078,29,++,Per7ume,Per7ume,Vidisco/nz Prod,14,47,2008,None
22120,11,NaN,Per7ume,Mudo,Vidisco,1,47,2010,None
22126,17,++,Adriana Lua,Adriana Lua,Espacial,3,47,2010,None
22211,12,12,NaN,Unorthodox Jukebox,Warner,2,47,2013,NaN
22222,23,23,Némanus,O melhor de Némanus,Espacial,19,47,2013,None
22254,25,17,Kátia Guerreiro,Até ao fim,Compact Records,2,47,2014,NaN
22276,17,NaN,Átoa,Idade dos inquietos,Universal,1,47,2015,NaN
22293,4,NaN,Força Suprema,E a união faz a força,Sony Music,1,47,2016,NaN
22313,24,12,Rodrigo Leão & Scott Matthew,Life Is Long,Universal,8,47,2016,NaN


In [230]:
newish.loc[newish['artist'] == 'Júlio Resende', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Paião', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Átoa', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Kátia Guerreiro', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Rodrigo Leão & Scott Matthew', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Per7ume', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Maria João & Mário Laginha', 'country'] = 'PT'

In [231]:
fix_nation = newish.loc[newish['country'].isnull()]
fix_nation.tail(30)

,rank,previous_week,artist,album,label,weeks_on_chart,calendar_week,year,country
21907,8,NaN,FKA Twigs,Magdalene,Popstock,1,46,2019,NaN
21920,21,21,C4pedro,The Gentleman,Sony Music,4,46,2019,NaN
21921,22,10,Soundtrack / Bruce Springsteen,Western Stars - Songs From The Film,Sony Music,3,46,2019,NaN
21923,24,NaN,The Script,Sunsets & Full Moons,Sony Music,1,46,2019,None
21958,29,23,Céline Dion & Anne Geddes,Miracle,Columbia,4,47,2004,NaN
22015,26,NaN,Clavezinha De Sol,A musica dos brinquedos III,Vidisco,0,47,2006,NaN
22032,13,++,Némanus,Dançando kizomba,Espacial,2,47,2007,None
22068,19,16,Madredeus & A Banda Cósmica,Metafonia,Farol Musica,4,47,2008,None
22126,17,++,Adriana Lua,Adriana Lua,Espacial,3,47,2010,None
22211,12,12,NaN,Unorthodox Jukebox,Warner,2,47,2013,NaN


In [232]:
newish.loc[newish['artist'] == 'Adriana Lua', 'country'] = 'BR'
newish.loc[newish['artist'] == 'Némanus', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Força Suprema', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Madredeus & A Banda Cósmica', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Clavezinha De Sol', 'country'] = 'PT'

newish.loc[newish['artist'] == 'The Script', 'country'] = 'IE'
newish.loc[newish['artist'] == 'Miley Cyrus', 'country'] = 'US'
newish.loc[newish['artist'] == 'FKA Twigs', 'country'] = 'GB'
newish.loc[newish['artist'] == 'Soundtrack / Bruce Springsteen', 'country'] = 'US'
newish.loc[newish['artist'] == 'Soundtrack / Vangelis', 'country'] = 'GR'

In [234]:
newish.isna().sum()

rank                 0
previous_week     2475
artist              55
album                0
label               57
weeks_on_chart       0
calendar_week        0
year                 0
country           1508
dtype: int64

In [236]:
fix_nation = newish.loc[newish['country'].isnull()]
fix_nation.head(30)

,rank,previous_week,artist,album,label,weeks_on_chart,calendar_week,year,country
312,13,14,NaN,Unorthodox Jukebox,Warner,8,1,2014,NaN
791,12,13,NaN,Unorthodox Jukebox,Warner,9,2,2014,NaN
1247,18,++,NaN,----------,NaN,0,3,2013,NaN
1248,19,++,NaN,----------,NaN,0,3,2013,NaN
1251,22,++,NaN,----------,NaN,0,3,2013,NaN
1253,24,++,NaN,----------,NaN,0,3,2013,NaN
1254,25,++,NaN,----------,NaN,0,3,2013,NaN
1258,29,++,NaN,----------,NaN,0,3,2013,NaN
1276,17,12,NaN,Unorthodox Jukebox,Warner,10,3,2014,NaN
1753,14,17,NaN,Unorthodox Jukebox,Warner,11,4,2014,NaN


In [238]:
newish.loc[newish['artist'] == 'Roberto Carlos e Caetano Veloso', 'country'] = 'BR'
newish.loc[newish['artist'] == 'Sérgio Godinho & Os Assessores', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Capicua & Pedro Geraldes', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Paulo Sousa', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Pedro Mestre', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Alex Amaral e Sandra Coelho', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Alexander Search', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Caelums Edge', 'country'] = 'PT'
newish.loc[newish['artist'] == 'União Das Tribos', 'country'] = 'PT'

newish.loc[newish['artist'] == 'Dream Theater', 'country'] = 'US'
newish.loc[newish['artist'] == 'Bring Me The Horizon', 'country'] = 'GB'
newish.loc[newish['artist'] == '5 Seconds of Summer', 'country'] = 'AU'

In [244]:
newish.loc[newish['artist'] == 'Luka [BR]', 'country'] = 'BR'
newish.loc[newish['artist'] == 'Xeg', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Ruca', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Mazgani', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Stereossauro', 'country'] = 'PT'
newish.loc[newish['artist'] == '3 Marias', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Padre Fabio De Melo', 'country'] = 'BR'
newish.loc[newish['artist'] == 'Paus [PT] 	', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Bruno Mendez', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Bruno Mars', 'country'] = 'US'

In [242]:
newish.loc[newish['album'] == 'Unorthodox Jukebox', 'artist'] = 'Bruno Mars'

In [248]:
newish.loc[newish['artist'] == 'Roberto Carlos e Caetano Veloso', 'country'] = 'BR'
newish.loc[newish['artist'] == 'Amelia Muge', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Ala Dos Namorados', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Kika', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Jimmy P', 'country'] = 'PT'
newish.loc[newish['artist'] == 'More Than A Thousand', 'country'] = 'PT'
newish.loc[newish['artist'] == 'HMB', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Músicas Infantis', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Mão Morta & Remix Ensemble', 'country'] = 'PT'

newish.loc[newish['artist'] == 'Car Seat Headrest', 'country'] = 'US'
newish.loc[newish['artist'] == 'Jack Johnson and Friends', 'country'] = 'US'
newish.loc[newish['artist'] == 'Atoms For Peace', 'country'] = 'GB'
newish.loc[newish['artist'] == 'Lodovica Comello', 'country'] = 'IT'
newish.loc[newish['artist'] == 'Massive Attack', 'country'] = 'GB'
newish.loc[newish['artist'] == 'Rag\'n\'Bone Man', 'country'] = 'GB'

In [250]:
newish.loc[newish['artist'] == 'Pedro Vaz', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Escola Do Panda', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Marta Ren & The Groovevelts', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Anaquim', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Capicua', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Sónia Araújo', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Francisco José', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Toranja', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Cindy Kat', 'country'] = 'PT'
newish.loc[newish['artist'] == 'EZ Special', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Manuel Fúria e Os Naufragos', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Carochinha', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Oquestrada', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Missy M', 'country'] = 'PT'

In [252]:
newish.isna().sum()

rank                 0
previous_week     2475
artist              38
album                0
label               57
weeks_on_chart       0
calendar_week        0
year                 0
country           1098
dtype: int64

In [262]:
newish.loc[newish['artist'] == '4Kids', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Adiafa', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Alfa [PT]', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Maria Lisboa', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Marta Peneda', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Marta Pereira Da Costa', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Maria João Pires / Claudio Abbad', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Aguias Vermelhas', 'country'] = 'PT'

In [269]:
newish.loc[newish['artist'] == 'Allana Parker', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Adriana [PT]', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Algodao', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Augusto Canário & Amigos', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Ana Bacalhau / Jorge Benvinda / Sérgio Godinho / Vitorino 	', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Banda do Panda', 'country'] = 'PT'
newish.loc[newish['artist'] == 'BandaLusa', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Bandalusa', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Antonio Pinho Vargas', 'country'] = 'PT'

newish.loc[newish['artist'] == 'Alt-J', 'country'] = 'GB'

In [271]:
newish.loc[newish['artist'] == 'Boy Teddy', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Canta Bahia', 'country'] = 'BR'
newish.loc[newish['artist'] == 'Caetano Veloso / Moreno Veloso / Zeca Veloso / Tom Veloso', 'country'] = 'BR'
newish.loc[newish['artist'] == 'Beth Gibbons And The Polish National Radio Symphony Orchestra / Krzysztof Penderecki', 'country'] = 'GB'
newish.loc[newish['artist'] == 'Ana Bacalhau / Jorge Benvinda / Sérgio Godinho / Vitorino', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Angel-O', 'country'] = 'PT'
newish.loc[newish['artist'] == 'BandaLusa', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Bandalusa', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Antonio Pinho Vargas', 'country'] = 'PT'

newish.loc[newish['artist'] == 'Ben Harper & The Blind Boys Of Alabama', 'country'] = 'US'
newish.loc[newish['artist'] == 'Ben Harper And Relentless7', 'country'] = 'US'
newish.loc[newish['artist'] == 'Ben Harper and Charlie Musselwhite', 'country'] = 'US'
newish.loc[newish['artist'] == 'Ben Harper', 'country'] = 'US'
newish.loc[newish['artist'] == 'Alice In Chains', 'country'] = 'US'

In [274]:
newish.loc[newish['artist'] == 'Cidinho e Doca', 'country'] = 'BR'
newish.loc[newish['artist'] == 'Carolina', 'country'] = 'BR'
newish.loc[newish['artist'] == 'Brass Wire Orchestra', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Tony De Matos', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Tiago Bettencourt & Mantha', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Tereza Salgueiro', 'country'] = 'PT'

newish.loc[newish['artist'] == 'Wynton Marsalis & Eric Clapton', 'country'] = 'US'
newish.loc[newish['artist'] == 'Within Temptation & The Metropole Orchestra', 'country'] = 'NL'

In [298]:
newish.loc[newish['artist'] == 'Fingertips', 'country'] = 'PT'
newish.loc[newish['artist'] == 'FingerTips', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Zé Amaro', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Gabriel O Pensador', 'country'] = 'BR'

newish.loc[newish['artist'] == 'Dário [PT]', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Frankie Chavez', 'country'] = 'PT'

In [290]:
newish.loc[newish['artist'] == 'Super Dragões', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Soundtrack / António Variações', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Sean Riley & The Slowriders', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Tony Bennett & Lady Gaga', 'country'] = 'US'

newish.loc[newish['artist'] == 'Thom Yorke', 'country'] = 'UK'
newish.loc[newish['artist'] == 'Soundtrack / AC/DC', 'country'] = 'AU'

In [305]:
newish.loc[newish['artist'] == 'Serafim & Companhia', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Soundtrack / António Variações', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Santamaria', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Supernada', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Sangre Ibérico', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Pintainho Amarelinho', 'country'] = 'PT'

newish.loc[newish['artist'] == 'Thom Yorke', 'country'] = 'UK'
newish.loc[newish['artist'] == 'Soundtrack / AC/DC', 'country'] = 'AU'

In [301]:
newish.loc[newish['artist'] == 'Eddie Vedder', 'country'] = 'US'
newish.loc[newish['artist'] == 'Gotye', 'country'] = 'AU'

In [308]:
newish.loc[newish['artist'] == 'Kizomba Brasil', 'country'] = 'BR'
newish.loc[newish['artist'] == 'Kizomba Sertanejo', 'country'] = 'BR'
newish.loc[newish['artist'] == 'Pintainho Amarelinho', 'country'] = 'PT'

In [317]:
newish.loc[newish['artist'] == 'Paus [PT]', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Peixe : Avião', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Liliane Marise', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Lena d\'Água', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Leandro E Leonardo', 'country'] = 'BR'

In [324]:
newish.loc[newish['artist'] == 'Muxima', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Mia Rose', 'country'] = 'PT'
newish.loc[newish['artist'] == 'M3 [PT]', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Luisa Sobral', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Vitorino & Janita Salome', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Luis Filipe Reis', 'country'] = 'PT'

In [339]:
fix_nation = newish.loc[newish['country'].isnull()].groupby(["artist"]).sum()
fix_nation.head(5)

,rank,weeks_on_chart,calendar_week,year
artist,,,,
Anamar,28,1,20,2013
André Rieu & His Johann Strauss Orchestra,55,3,41,4022
André Viamonte,48,3,89,4038
Anohni,90,12,92,8064
Aretha Franklin with The Royal Philharmonic Orchestra,42,5,81,4035
...,...,...,...,...
Milton Soares,27,1,23,2005
Mirri Lobo,28,1,10,2019
Monjes Budistas,98,0,123,12036


In [333]:
newish.loc[newish['artist'] == 'Ladrões Do Tempo', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Lingua Franca', 'country'] = 'BR/PT'
newish.loc[newish['artist'] == 'Mesa', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Miguel Angelo', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Zé do Pipo', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Mastiksoul', 'country'] = 'PT'

newish.loc[newish['artist'] == 'Maria João Pires / Claudio Abbad', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Martina Stoessel', 'country'] = 'AG'
newish.loc[newish['artist'] == 'Herman José', 'country'] = 'PT'
newish.loc[newish['artist'] == 'Israel Kamakawiwo\'ole', 'country'] = 'US'
newish.loc[newish['artist'] == 'Ivete / Gil / Caetano', 'country'] = 'BR'
newish.loc[newish['artist'] == 'Luis Filipe Reis', 'country'] = 'PT'

In [342]:
newish.to_csv('newish.csv')